In [ ]:
import pandas as pd

In [29]:
# Gerekli kütüphaneleri yüklüyoruz
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
import time
import pandas as pd

# Chrome tarayıcıyı driverla başlatıyoruz
options = uc.ChromeOptions()
options.headless = False  # Başsız modda çalıştırmak için True yapabilirsiniz
options.add_argument("--no-sandbox")  # Sandbox modunu devre dışı bırakır
options.add_argument("--disable-dev-shm-usage")  # Paylaşılan belleği devre dışı bırakır

# Driver başlat
driver = uc.Chrome(options=options)

# URL'yi tanımlayıp bekliyoruz
url = "https://widgets.weforum.org/future-of-jobs-2025/scorecards/score-card/economy/TUR.json"
driver.get(url)

# Yüklenme süresi için bekleme ekliyoruz
time.sleep(3)

# Sayfanın içeriğini alıyoruz
html_content = driver.page_source

# BeautifulSoup ile HTML içeriğini işliyoruz
soup = BeautifulSoup(html_content, "html.parser")


SessionNotCreatedException: Message: session not created: cannot connect to chrome at 127.0.0.1:61592
from session not created: This version of ChromeDriver only supports Chrome version 132
Current browser version is 131.0.6778.265
Stacktrace:
	GetHandleVerifier [0x00BD0A13+25091]
	(No symbol) [0x00B5A584]
	(No symbol) [0x00A3B3B3]
	(No symbol) [0x00A70A62]
	(No symbol) [0x00A6FA99]
	(No symbol) [0x00A664CF]
	(No symbol) [0x00A66317]
	(No symbol) [0x00AAB05A]
	(No symbol) [0x00AAABCA]
	(No symbol) [0x00AA1C46]
	(No symbol) [0x00A73175]
	(No symbol) [0x00A742FD]
	GetHandleVerifier [0x00EC6493+3128451]
	GetHandleVerifier [0x00ED994B+3207483]
	GetHandleVerifier [0x00ED45F2+3186146]
	GetHandleVerifier [0x00C670C0+641200]
	(No symbol) [0x00B636BD]
	(No symbol) [0x00B60738]
	(No symbol) [0x00B608D6]
	(No symbol) [0x00B53040]
	BaseThreadInitThunk [0x76E05D49+25]
	RtlInitializeExceptionChain [0x7709CEBB+107]
	RtlGetAppContainerNamedObjectPath [0x7709CE41+561]


In [36]:
with open('https://widgets.weforum.org/future-of-jobs-2025/scorecards/score-card/economy/ARG.json') as datafile:
    for line in datafile:
        data = json.loads(line)
    retail = pd.DataFrame(data)

OSError: [Errno 22] Invalid argument: 'https://widgets.weforum.org/future-of-jobs-2025/scorecards/score-card/economy/ARG.json'

In [32]:
import json
import pandas as pd

# JSON dosyasını okuyup yapısını inceleyin
with open("TUR.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# JSON verisini yazdır
print(json.dumps(data, indent=4))

# Üst düzey bilgileri al
country_data = {
    "Country": data["topSection"]["title"],
    "ScoreCardType": data["topSection"]["scoreCardTypeLabel"],
    "WorkingAgePopulation": data["topSection"]["globalInfo"]["value"],
}

# Bölümleri ve alt bölümleri işleme
sections_data = []
for section in data.get("sections", []):
    section_title = section.get("title", "")
    for sub_section in section.get("subSections", []):
        sub_section_title = sub_section.get("title", "")
        for entry in sub_section.get("entries", []):
            sections_data.append({
                "Section": section_title,
                "SubSection": sub_section_title,
                "Title": entry.get("title", ""),
                "Subtitle": entry.get("subtitle", ""),
                "PrimaryPercentage": entry.get("primaryPercentage", None),
                "SecondaryPercentage": entry.get("secondaryPercentage", None),
                "SecondaryLabel": entry.get("secondaryLabel", ""),
            })

# DataFrame'e dönüştür
country_df = pd.DataFrame([country_data])
sections_df = pd.DataFrame(sections_data)

# Tabloları birleştirme
result_df = pd.concat([country_df, sections_df], axis=0, ignore_index=True)

# DataFrame'i görselleştir
result_df


{
    "id": "TUR",
    "type": "economy",
    "topSection": {
        "title": "T\u00fcrkiye",
        "scoreCardTypeLabel": "Economy Profile",
        "globalInfo": {
            "value": 53.59,
            "label": "Working Age Population (Millions)",
            "format": "float"
        }
    },
    "sections": [
        {
            "id": "jobs-skills-outlook",
            "title": "Jobs and Skills outlook",
            "subSections": [
                {
                    "id": "1. Jobs and Skills outlook",
                    "title": "Labour-market churn",
                    "subTitle": "",
                    "visualModelType": "summary",
                    "visualModelSubType": "summary-simple",
                    "entries": [
                        {
                            "primaryPercentage": 29,
                            "secondaryLabel": "Global",
                            "secondaryPercentage": 22,
                            "title": "Labour-market churn"

,Country,ScoreCardType,WorkingAgePopulation,Section,SubSection,Title,Subtitle,PrimaryPercentage,SecondaryPercentage,SecondaryLabel
0,Türkiye,Economy Profile,53.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Jobs and Skills outlook,Labour-market churn,Labour-market churn,Five-year structural labour-force churn,29.0,22.0,Global
2,NaN,NaN,NaN,Jobs and Skills outlook,Labour-market churn,Skill disruption,Shares of core skills which will change,44.0,39.0,Global
3,NaN,NaN,NaN,Jobs and Skills outlook,Labour-market churn,Organizations with DEI priorities,Share of organizations with DEI priorities,88.0,83.0,Global
4,NaN,NaN,NaN,Jobs and Skills outlook,Labour-market churn,AI exposure,Share of organizations running AI programmes,94.0,88.0,Global
...,...,...,...,...,...,...,...,...,...,...
73,NaN,NaN,NaN,DEI Actions,,Run comprehensive DEI training for managers an...,Run comprehensive DEI training for managers an...,54.0,51.0,Global
74,NaN,NaN,NaN,DEI Actions,,Pay equity reviews and salary audits,Pay equity reviews and salary audits,46.0,39.0,Global
75,NaN,NaN,NaN,AI Strategy,,Hiring new people with skills to design AI too...,Hiring new people with skills to design AI too...,78.0,69.0,Global
76,NaN,NaN,NaN,AI Strategy,,Reskilling and upskilling your existing workfo...,Reskilling and upskilling your existing workfo...,70.0,77.0,Global


In [39]:
with open("https://widgets.weforum.org/future-of-jobs-2025/scorecards/score-card/economy/ARG.json", "r", encoding="utf-8") as file:
    data = json.load(file)
data

OSError: [Errno 22] Invalid argument: 'https://widgets.weforum.org/future-of-jobs-2025/scorecards/score-card/economy/ARG.json'